In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - apo_mock_test_class
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Test the APOGEEMock class
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, glob, subprocess, warnings, dill as pickle, time
from astropy import units as apu

## Plotting
from matplotlib import pyplot as plt
import corner

## galpy
from galpy import orbit
from galpy import potential

## APOGEE, isochrones, dustmaps
from apogee import select as apsel
from isodist import Z2FEH,FEH2Z
import mwdust

## scipy
import scipy.integrate
import scipy.stats

## Mock package
import apomock

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup
%matplotlib inline
plt.style.use('../../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# Reload
import importlib
importlib.reload(apomock)

### Test inititialization and basic class properties

In [ ]:
# denspot correct, ro/vo correct
denspot = potential.HernquistPotential()
test_ro = 8.1
test_vo = 222.
mock = apomock.APOGEEMock(denspot=denspot,ro=test_ro,vo=test_vo)
assert isinstance(mock._denspot, potential.HernquistPotential)
assert mock._ro == test_ro
assert mock._vo == test_vo

# ro/vo in astropy units
test_ro = 800.*apu.pc
test_vo = 220000.*apu.m/apu.s
mock = apomock.APOGEEMock(denspot=denspot, ro=test_ro, vo=test_vo)
assert mock._ro == test_ro.to(apu.kpc).value
assert mock._vo == test_vo.to(apu.km/apu.s).value

# ro/vo inheritance from denspot
denspot = potential.HernquistPotential(amp=1e7*apu.M_sun,a=20*apu.kpc,
                                       ro=ro,vo=vo)
mock = apomock.APOGEEMock(denspot=denspot)
assert mock._ro == denspot._ro
assert mock._vo == denspot._vo

### Test mass sampling

In [ ]:
denspot = potential.HernquistPotential()
mock = apomock.APOGEEMock(denspot=denspot)
iso_filename = filename = '/home/lane/data/software/isodist/parsec1.2/2mass-spitzer-wise-0.0001-z-0.0060-1e10-age-1.4e10.dat'
iso_z = 0.0015
iso_log_age = np.log10(1.1e10)
mock.load_parsec_isochrone(iso_filename, z=iso_z, log_age=iso_log_age)

In [ ]:
m_tot = 1e4*apu.M_sun
m_min = 0.1*apu.M_sun
m_max = 0.8*apu.M_sun
mock.sample_masses

In [ ]:
imf_types = ['chabrier','kroupa']
imf_fns = [apomock.util.util.chabrier_imf,apomock.util.util.kroupa_imf]
m_min = 0.1
m_max = 0.9
m_tot = 1e6

imf_lims = np.array([0.4,0.5,0.6,0.7,0.8])

for i in range(len(imf_types)):
    
    mock.sample_masses(m_tot, imf_type=imf_types[i], m_min=m_min, m_max=m_max, 
                       force_resample=True)
    ms = mock.masses
    assert np.min(ms) > m_min
    assert np.max(ms) < m_max
    assert np.fabs(np.sum(ms)-m_tot)/m_tot < 1e-3
    
    m_lim_int_callable = lambda x: x*imf_fns[i](x)
    imf_mass_tot = scipy.integrate.quad(m_lim_int_callable, a=m_min, 
                                        b=m_max)[0]
    
    for j in range(len(imf_lims)):
        imf_lim_mass = scipy.integrate.quad(m_lim_int_callable, a=m_min, 
                                            b=imf_lims[j])[0]
        imf_lim_mass_frac = imf_lim_mass/imf_mass_tot
        sample_lim_mass_frac = np.sum(ms[ms<imf_lims[j]])/m_tot
        assert np.fabs(sample_lim_mass_frac-imf_lim_mass_frac)/imf_lim_mass_frac\
            < 0.01
                       

## Test position sampling
Do it in shells

In [ ]:
ro,vo = 8.,220.
denspots = [potential.HernquistPotential(),
            potential.PowerSphericalPotential(alpha=2.),
            potential.PowerSphericalPotential(alpha=3.1),
            potential.PowerSphericalPotentialwCutoff(alpha=2.5,rc=3.),
            potential.PowerSphericalPotentialwCutoff(alpha=3.5,rc=5.),
            potential.NFWPotential()
           ]

rmin = 2./ro
rmax = 70./ro
rlims = np.arange(10.,70.,10.)/ro

for denspot in denspots:
    print(denspot)
    mock = apomock.APOGEEMock(denspot=denspot)
    _n_fake_masses = int(1e6)
    _fake_masses = np.ones(_n_fake_masses) # Hack
    mock.masses = _fake_masses
    mock.sample_positions(r_min=2./ro,r_max=70./ro)
    rs = mock.orbs.r(use_physical=False)
    pot_mtot = potential.mass(denspot,rmax,use_physical=False)-\
        potential.mass(denspot,rmin,use_physical=False)
    for rlim in rlims:
        pot_mfrac = (potential.mass(denspot,rlim,use_physical=False)-\
            potential.mass(denspot,rmin,use_physical=False))/pot_mtot
        sample_mfrac = len(np.where(rs < rlim)[0])/_n_fake_masses
        print(np.fabs(pot_mfrac-sample_mfrac)/pot_mfrac)
        assert np.fabs(pot_mfrac-sample_mfrac)/pot_mfrac < 5e-3,\
            'sample mass within lim does not match denspot mass within lim'

## Test position sampling
Take a more detailed look

In [ ]:
ro,vo = 8.,220.
denspots = [potential.HernquistPotential(ro=ro,vo=vo),
            potential.PowerSphericalPotential(alpha=2.,ro=ro,vo=vo),
            potential.PowerSphericalPotential(alpha=3.1,ro=ro,vo=vo),
            potential.PowerSphericalPotentialwCutoff(alpha=2.5,rc=3.,ro=ro,vo=vo),
            potential.PowerSphericalPotentialwCutoff(alpha=3.5,rc=5.,ro=ro,vo=vo),
            potential.NFWPotential(ro=ro,vo=vo)
           ]

rmin = 2./ro
rmax = 70./ro

for i in range(len(denspots)):
    
    # if i > 1: continue
    
    fig = plt.figure(figsize=(20,3))
    axs = fig.subplots(nrows=1, ncols=5)
    
    # Make sampled data
    mock = apomock.APOGEEMock(denspot=denspots[i],ro=ro,vo=vo)
    n_samples = int(1e6)
    fake_masses = np.ones(n_samples) # Hack
    mock.masses = fake_masses
    mock.sample_positions(r_min=rmin,r_max=rmax)
    sample_rs = mock.orbs.r(use_physical=False)
    sample_rs_sort = np.sort(sample_rs)
    sample_mass_cml = np.arange(0,n_samples)/n_samples
    n_sample_dens = 20
    sample_dens = np.zeros(n_sample_dens)
    sample_dens_redge = np.linspace(rmin, rmax, n_sample_dens+1)
    sample_dens_rcent = (sample_dens_redge[1:]+sample_dens_redge[:-1])/2
    for j in range(n_sample_dens):
        n_in_bin = len(np.where((sample_rs>sample_dens_redge[j]) &\
                                (sample_rs<sample_dens_redge[j+1]))[0])
        bin_vol = sample_dens_redge[j+1]**3 - sample_dens_redge[j]**3
        sample_dens[j] = n_in_bin / bin_vol
    sample_dens = sample_dens/sample_dens[0]
    
    pot_dens_rs = np.linspace(sample_dens_rcent[0], rmax, 100)
    pot_mass_rs = np.linspace(rmin, rmax, 100)
    pot_dens = potential.evaluateDensities(denspots[i], pot_dens_rs, 0, use_physical=False)
    try:
        pot_mass = potential.mass(denspots[i], pot_mass_rs, use_physical=False)
    except (TypeError,AttributeError):
        pot_mass = np.array([potential.mass(denspots[i], pot_r, use_physical=False)\
                             for pot_r in pot_mass_rs])
    pot_dens = pot_dens/potential.evaluateDensities(denspots[i], pot_dens_rs[0], 0, use_physical=False)
    pot_mass = (pot_mass-potential.mass(denspots[i], rmin, use_physical=False))/\
               (potential.mass(denspots[i], rmax, use_physical=False)-\
                potential.mass(denspots[i], rmin, use_physical=False))
        
    axs[0].plot(np.log10(pot_dens_rs), np.log10(pot_dens), color='Black', linewidth=2.)
    axs[0].scatter(np.log10(sample_dens_rcent), 
                   np.log10(sample_dens), color='DodgerBlue')
    
    axs[1].plot(np.log10(pot_mass_rs), pot_mass, color='Black', linewidth=2.)
    mass_skip = int(n_samples/50)
    axs[1].scatter(np.log10(sample_rs_sort[::mass_skip]), 
                   sample_mass_cml[::mass_skip], 
                   color='DodgerBlue', linestyle='dashed')
    
    axs[2].hist(mock.orbs.phi(use_physical=True), bins=50, range=(0,2*np.pi), 
               histtype='step', density=True)
    
    axs[3].hist(mock.orbs.theta(use_physical=True), bins=50, range=(0,np.pi),
               histtype='step', density=True)
    
    axs[0].set_xlabel('log r')
    axs[0].set_ylabel('density [normalized]')
    
    axs[1].set_xlabel('log r')
    axs[1].set_ylabel('cumulative mass profile')
    
    axs[2].set_xlabel(r'$\phi$')
    axs[2].set_ylabel('N')
    
    axs[3].set_xlabel(r'$\theta$')
    axs[3].set_ylabel('N')
    
    fig.suptitle(denspots[i].__class__)
    
    fig.show()


Specifically examine the power law potential. Fit the sampled density profiles

In [ ]:
alpha = 2.
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)

rmins = np.array([4.,2.,1.,0.5,0.1])/ro

for i in range(len(rmins)):
    
    fig = plt.figure(figsize=(10,3))
    axs = fig.subplots(nrows=1, ncols=2)
        
    rmin = rmins[i]
    rmax = 20.*rmin

    # Make sampled data
    mock = apomock.APOGEEMock(denspot=denspot,ro=ro,vo=vo)
    n_samples = int(1e6)
    fake_masses = np.ones(n_samples) # Hack
    mock.masses = fake_masses
    mock.sample_positions(r_min=rmin,r_max=rmax)
    sample_rs = mock.orbs.r(use_physical=False)
    sample_rs_sort = np.sort(sample_rs)
    sample_mass_cml = np.arange(0,n_samples)/n_samples

    n_sample_dens = 20
    sample_dens = np.zeros(n_sample_dens)
    sample_dens_redge = np.logspace(np.log10(rmin), np.log10(rmax), n_sample_dens+1)
    sample_dens_rcent = 10**((np.log10(sample_dens_redge[1:])+\
                              np.log10(sample_dens_redge[:-1]))/2.)

    # sample_dens_rcent = (sample_dens_redge[1:]+sample_dens_redge[:-1])/2
    for j in range(n_sample_dens):
        n_in_bin = len(np.where((sample_rs>sample_dens_redge[j]) &\
                                (sample_rs<sample_dens_redge[j+1]))[0])
        bin_vol = sample_dens_redge[j+1]**3 - sample_dens_redge[j]**3
        sample_dens[j] = n_in_bin / bin_vol
    sample_dens = sample_dens/sample_dens[0]

    # Make potential data for comparison
    pot_dens_rs = np.logspace(np.log10(sample_dens_rcent[0]), np.log10(rmax), 100)
    pot_mass_rs = np.logspace(np.log10(rmin), np.log10(rmax), 100)
    pot_dens = potential.evaluateDensities(denspot, pot_dens_rs, 0, use_physical=False)
    try:
        pot_mass = potential.mass(denspot, pot_mass_rs, use_physical=False)
    except (TypeError,AttributeError):
        pot_mass = np.array([potential.mass(denspot, pot_r, use_physical=False)\
                             for pot_r in pot_mass_rs])
    pot_dens = pot_dens/potential.evaluateDensities(denspot, pot_dens_rs[0], 0, use_physical=False)
    pot_mass = (pot_mass-potential.mass(denspot, rmin, use_physical=False))/\
               (potential.mass(denspot, rmax, use_physical=False)-\
                potential.mass(denspot, rmin, use_physical=False))

    axs[0].plot(np.log10(pot_dens_rs), np.log10(pot_dens), color='Black', linewidth=2.)
    axs[0].scatter(np.log10(sample_dens_rcent), 
                   np.log10(sample_dens), color='DodgerBlue')
    
    fit = scipy.stats.linregress(np.log10(sample_dens_rcent), np.log10(sample_dens))
    axs[0].annotate('alpha: '+str(-1*fit.slope), xy=(0.5,0.9), xycoords='axes fraction')

    axs[1].plot(np.log10(pot_mass_rs), pot_mass, color='Black', linewidth=2.)
    mass_skip = int(n_samples/50)
    axs[1].scatter(np.log10(sample_rs_sort[::mass_skip]), 
                   sample_mass_cml[::mass_skip], 
                   color='DodgerBlue', linestyle='dashed')
    
    axs[0].set_xlabel('log r')
    axs[0].set_ylabel('density [normalized]')
    
    axs[1].set_xlabel('log r')
    axs[1].set_ylabel('cumulative mass profile')
    
    fig.suptitle('rmin: '+str(rmins[i]))
    
    fig.show()

In [ ]:
alpha = 4.
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)

rmins = np.array([4.,2.,1.,0.5,0.1])/ro

for i in range(len(rmins)):
    
    fig = plt.figure(figsize=(10,3))
    axs = fig.subplots(nrows=1, ncols=2)
        
    rmin = rmins[i]
    rmax = 70./ro

    # Make sampled data
    mock = apomock.APOGEEMock(denspot=denspot,ro=ro,vo=vo)
    n_samples = int(1e6)
    fake_masses = np.ones(n_samples) # Hack
    mock.masses = fake_masses
    mock.sample_positions(r_min=rmin,r_max=rmax)
    sample_rs = mock.orbs.r(use_physical=False)
    sample_rs_sort = np.sort(sample_rs)
    sample_mass_cml = np.arange(0,n_samples)/n_samples

    n_sample_dens = 20
    sample_dens = np.zeros(n_sample_dens)
    sample_dens_redge = np.logspace(np.log10(rmin), np.log10(rmax), n_sample_dens+1)
    sample_dens_rcent = 10**((np.log10(sample_dens_redge[1:])+\
                              np.log10(sample_dens_redge[:-1]))/2.)

    # sample_dens_rcent = (sample_dens_redge[1:]+sample_dens_redge[:-1])/2
    for j in range(n_sample_dens):
        n_in_bin = len(np.where((sample_rs>sample_dens_redge[j]) &\
                                (sample_rs<sample_dens_redge[j+1]))[0])
        bin_vol = sample_dens_redge[j+1]**3 - sample_dens_redge[j]**3
        sample_dens[j] = n_in_bin / bin_vol
    sample_dens = sample_dens/sample_dens[0]

    # Make potential data for comparison
    pot_dens_rs = np.logspace(np.log10(sample_dens_rcent[0]), np.log10(rmax), 100)
    pot_mass_rs = np.logspace(np.log10(rmin), np.log10(rmax), 100)
    pot_dens = potential.evaluateDensities(denspot, pot_dens_rs, 0, use_physical=False)
    try:
        pot_mass = potential.mass(denspot, pot_mass_rs, use_physical=False)
    except (TypeError,AttributeError):
        pot_mass = np.array([potential.mass(denspot, pot_r, use_physical=False)\
                             for pot_r in pot_mass_rs])
    pot_dens = pot_dens/potential.evaluateDensities(denspot, pot_dens_rs[0], 0, use_physical=False)
    pot_mass = (pot_mass-potential.mass(denspot, rmin, use_physical=False))/\
               (potential.mass(denspot, rmax, use_physical=False)-\
                potential.mass(denspot, rmin, use_physical=False))

    axs[0].plot(np.log10(pot_dens_rs), np.log10(pot_dens), color='Black', linewidth=2.)
    axs[0].scatter(np.log10(sample_dens_rcent), 
                   np.log10(sample_dens), color='DodgerBlue')
    
    fit = scipy.stats.linregress(np.log10(sample_dens_rcent), np.log10(sample_dens))
    axs[0].annotate('alpha: '+str(-1*fit.slope), xy=(0.5,0.9), xycoords='axes fraction')

    axs[1].plot(np.log10(pot_mass_rs), pot_mass, color='Black', linewidth=2.)
    mass_skip = int(n_samples/50)
    axs[1].scatter(np.log10(sample_rs_sort[::mass_skip]), 
                   sample_mass_cml[::mass_skip], 
                   color='DodgerBlue', linestyle='dashed')
    
    axs[0].set_xlabel('log r')
    axs[0].set_ylabel('density [normalized]')
    
    axs[1].set_xlabel('log r')
    axs[1].set_ylabel('cumulative mass profile')
    
    fig.suptitle('rmin: '+str(rmins[i]))
    
    fig.show()

Check to make sure all samples are uncorrelated

In [ ]:
ro,vo = 8.,220.
denspots = [potential.HernquistPotential(ro=ro,vo=vo),
            potential.PowerSphericalPotential(alpha=2.,ro=ro,vo=vo),
            potential.PowerSphericalPotential(alpha=3.1,ro=ro,vo=vo),
            potential.PowerSphericalPotentialwCutoff(alpha=2.5,rc=3.,ro=ro,vo=vo),
            potential.PowerSphericalPotentialwCutoff(alpha=3.5,rc=5.,ro=ro,vo=vo),
            potential.NFWPotential(ro=ro,vo=vo)
           ]

rmin = 2./ro
rmax = 70./ro

for i in range(len(denspots)):
    
    # if i > 1: continue
    
    # Make sampled data
    mock = apomock.APOGEEMock(denspot=denspots[i],ro=ro,vo=vo)
    n_samples = int(1e6)
    fake_masses = np.ones(n_samples) # Hack
    mock.masses = fake_masses
    mock.sample_positions(r_min=rmin,r_max=rmax)
    sample_rs = mock.orbs.r(use_physical=False)
    sample_phis = mock.orbs.phi(use_physical=False)
    sample_thetas = mock.orbs.theta(use_physical=False)
    
    skip = 10
    fig = corner.corner(np.vstack([sample_rs,sample_phis,sample_thetas])[:,::skip].T,
                        labels = ['r',r'$\phi$',r'$\theta$'], show_labels=True)
    fig.show()
    